In [33]:
!pip install numpy==1.26.4
!pip install datasets==3.2.0
!pip install transformers==4.11.3
!pip install huggingface_hub==0.27.1
!pip install evaluate==0.4.3
!pip install torchaudio
!pip install librosa
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 7.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninsta

In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset, Audio, ClassLabel
import evaluate
import pandas as pd
from IPython.display import display, HTML
import IPython.display as ipd
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Trainer, TrainingArguments
import numpy as np
import torch
import wandb

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import os
import json
import random
import re

2025-01-31 12:15:25.749885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738314925.766035  171772 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738314925.771101  171772 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 12:15:25.788238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
notebook_login()

In [3]:
wandb.login()

wandb: Currently logged in as: vladimirshilonosov2 (vladimirshilonosov2-itmo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
### Скачивание speech to text датасета, с фильтром на русский язык

In [5]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ru", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "ru", split="test")

In [7]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,"А теперь, как я вижу, слова просит уважаемый посол Соединенных Штатов.",
1,Сколь много в этом коварства!,
2,Сейчас я предоставляю слово представителю Перу.,
3,Первым оратором в списке на сегодня является Ирландия от имени Европейского союза.,
4,Вот мы для Агафьи Михайловны послали за земским доктором.,
5,"Письменные принадлежности, до которых Алексей Александрович был охотник, были необыкновенно хороши.",
6,Смерть отсталым измереньям!,
7,Кто-то должен его заменить.,
8,"Он сел писать письмо, но через окно увидел идущего по дорожке красноармейца.",
9,"Позиция, занятая нашими вооруженными силами, останется в истории.",


In [4]:
### Удаление нефонетических символов

In [9]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\’\…\(\)\—\−\„\‑\–\«\»]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

<>:1: SyntaxWarning: invalid escape sequence '\,'
<>:1: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipykernel_171772/3899105336.py:1: SyntaxWarning: invalid escape sequence '\,'
  chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\’\…\(\)\—\−\„\‑\–\«\»]'


In [10]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [11]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,только одно конкретное предложение получило поддержку большинства государствчленов из всех региональных групп,
1,еще лучше смотреть в таз с водой,
2,элементы питания,
3,президента грузии господина михаила саакашвили сопровождают в зал генеральной ассамблеи,
4,в документе справедливо осуждается вербовка детей и использование молодых людей в вооруженных конфликтах,
5,спасибо вам посол,
6,первая касается обеспечения мира и безопасности,
7,вы можете проверить по часам,
8,полагаем что такому подходу можно было бы уделить в докладе даже больше внимания,
9,широким признанием пользуется необходимость укрепления институциональной памяти канцелярии председателя генеральной ассамблеи,


In [12]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/36580 [00:00<?, ? examples/s]

Map:   0%|          | 0/10203 [00:00<?, ? examples/s]

In [14]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [15]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'k': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'r': 15,
 's': 16,
 't': 17,
 'x': 18,
 'z': 19,
 'а': 20,
 'б': 21,
 'в': 22,
 'г': 23,
 'д': 24,
 'е': 25,
 'ж': 26,
 'з': 27,
 'и': 28,
 'й': 29,
 'к': 30,
 'л': 31,
 'м': 32,
 'н': 33,
 'о': 34,
 'п': 35,
 'р': 36,
 'с': 37,
 'т': 38,
 'у': 39,
 'ф': 40,
 'х': 41,
 'ц': 42,
 'ч': 43,
 'ш': 44,
 'щ': 45,
 'ъ': 46,
 'ы': 47,
 'ь': 48,
 'э': 49,
 'ю': 50,
 'я': 51,
 'ё': 52}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

55

In [18]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
repo_name = "wav2vec2-large-xls-r-300m-ru"

In [20]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [5]:
### Сохранение токенайзера в hf репозитории

In [21]:
tokenizer.push_to_hub(repo_name)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru/commit/8d18d86e0ef9d3eec9c8ff0601638856907c99c1', commit_message='Upload tokenizer', commit_description='', oid='8d18d86e0ef9d3eec9c8ff0601638856907c99c1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru', endpoint='https://huggingface.co', repo_type='model', repo_id='NLPVladimir/wav2vec2-large-xls-r-300m-ru'), pr_revision=None, pr_num=None)

In [22]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [23]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [25]:
rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

позвольте мне также поблагодарить секретаря комиссии господина сергея чернявского за его неизменную помощь


In [26]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: в то же время президент альбашир угрожает международному сообществу возмездием и новыми преступлениями
Input array shape: (138624,)
Sampling rate: 16000


In [27]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [6]:
### Предобработка датасета, подготовка к обучению

In [28]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [29]:
@dataclass
class DataCollatorCTCWithPadding:
    
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        batch['input_values'] = batch['input_values']

        return batch


In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
wer_metric = evaluate.load("wer")

In [8]:
### Рассчёт метрик

In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
### Инициализация предобученной модели speech to text

In [33]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model.freeze_feature_encoder()

In [35]:
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=False,
  per_device_train_batch_size=6,
  gradient_accumulation_steps=5,
  eval_strategy="steps",
  num_train_epochs=7,
  gradient_checkpointing=False,
  fp16=True,
  save_steps=500,
  eval_steps=10,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
  run_name='wav2vec2-large-xls-fine-tuning-ru',
  report_to="wandb",
  hub_model_id="NLPVladimir/wav2vec2-large-xls-r-300m-ru"
)

In [36]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    processing_class=processor.feature_extractor,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
### Дообучение модели на русский язык

In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,4.658200,0.805458,0.783762
1000,0.434200,0.308535,0.349233
1500,0.257200,0.243938,0.269182
2000,0.173100,0.206996,0.226576


/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warn

In [11]:
### Сохранение модели

In [39]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru/commit/01b0d51d79386b9bd9a53ba2fce0e489ac01b31d', commit_message='End of training', commit_description='', oid='01b0d51d79386b9bd9a53ba2fce0e489ac01b31d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru', endpoint='https://huggingface.co', repo_type='model', repo_id='NLPVladimir/wav2vec2-large-xls-r-300m-ru'), pr_revision=None, pr_num=None)